In [9]:
pip install langchain


[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [10]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Verify the variables are loaded (optional)
print(f"LANGSMITH_TRACING: {os.getenv('LANGSMITH_TRACING')}")
print(f"LANGSMITH_API_KEY: {'*' * 20 if os.getenv('LANGSMITH_API_KEY') else 'Not set'}")

LANGSMITH_TRACING: true
LANGSMITH_API_KEY: ********************


In [ ]:
import os
from langchain_openai import ChatOpenAI

os.environ["OPENAI_API_KEY"] = ""
model = ChatOpenAI(model="gpt-4.1")

In [12]:
pip install -U "langchain[openai]"


[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [13]:
from langchain.tools import tool

@tool
def create_calendar_event(
    title: str,
    start_time: str,       # ISO format: "2024-01-15T14:00:00"
    end_time: str,         # ISO format: "2024-01-15T15:00:00"
    attendees: list[str],  # email addresses
    location: str = ""
) -> str:
    """Create a calendar event. Requires exact ISO datetime format."""
    # Stub: In practice, this would call Google Calendar API, Outlook API, etc.
    return f"Event created: {title} from {start_time} to {end_time} with {len(attendees)} attendees"


@tool
def send_email(
    to: list[str],  # email addresses
    subject: str,
    body: str,
    cc: list[str] = []
) -> str:
    """Send an email via email API. Requires properly formatted addresses."""
    # Stub: In practice, this would call SendGrid, Gmail API, etc.
    return f"Email sent to {', '.join(to)} - Subject: {subject}"


@tool
def get_available_time_slots(
    attendees: list[str],
    date: str,  # ISO format: "2024-01-15"
    duration_minutes: int
) -> list[str]:
    """Check calendar availability for given attendees on a specific date."""
    # Stub: In practice, this would query calendar APIs
    return ["09:00", "14:00", "16:00"]

In [14]:
from langchain.agents import create_agent


CALENDAR_AGENT_PROMPT = (
    "You are a calendar scheduling assistant. "
    "Parse natural language scheduling requests (e.g., 'next Tuesday at 2pm') "
    "into proper ISO datetime formats. "
    "Use get_available_time_slots to check availability when needed. "
    "Use create_calendar_event to schedule events. "
    "Always confirm what was scheduled in your final response."
)

calendar_agent = create_agent(
    model,
    tools=[create_calendar_event, get_available_time_slots],
    system_prompt=CALENDAR_AGENT_PROMPT,
)

In [15]:
query = "Schedule a team meeting next Tuesday at 2pm for 1 hour"

for step in calendar_agent.stream(
    {"messages": [{"role": "user", "content": query}]}
):
    for update in step.values():
        for message in update.get("messages", []):
            message.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  get_available_time_slots (call_Yp1RgHKNOpM7RLZAPT6f06eo)
 Call ID: call_Yp1RgHKNOpM7RLZAPT6f06eo
  Args:
    attendees: []
    date: 2024-06-18
    duration_minutes: 60
================================= Tool Message =================================
Name: get_available_time_slots

["09:00", "14:00", "16:00"]
================================== Ai Message ==================================
Tool Calls:
  create_calendar_event (call_7lZpr6avGbNw0X4gVZD8Syk6)
 Call ID: call_7lZpr6avGbNw0X4gVZD8Syk6
  Args:
    title: Team Meeting
    start_time: 2024-06-18T14:00:00
    end_time: 2024-06-18T15:00:00
    attendees: []
================================= Tool Message =================================
Name: create_calendar_event

Event created: Team Meeting from 2024-06-18T14:00:00 to 2024-06-18T15:00:00 with 0 attendees
================================== Ai Message ==================================

T

In [16]:
EMAIL_AGENT_PROMPT = (
    "You are an email assistant. "
    "Compose professional emails based on natural language requests. "
    "Extract recipient information and craft appropriate subject lines and body text. "
    "Use send_email to send the message. "
    "Always confirm what was sent in your final response."
)

email_agent = create_agent(
    model,
    tools=[send_email],
    system_prompt=EMAIL_AGENT_PROMPT,
)

In [17]:
query = "Send the design team a reminder about reviewing the new mockups"

for step in email_agent.stream(
    {"messages": [{"role": "user", "content": query}]}
):
    for update in step.values():
        for message in update.get("messages", []):
            message.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  send_email (call_zwJDObhnFQ7k5yojjlbnt3L8)
 Call ID: call_zwJDObhnFQ7k5yojjlbnt3L8
  Args:
    to: ['design-team@example.com']
    subject: Reminder: Please Review the New Mockups
    body: Hello Team,

This is a friendly reminder to review the new mockups at your earliest convenience. Your feedback is important to ensure we move forward smoothly with the next stages.

Thank you and please share your thoughts once you've had a chance to review.

Best regards,
================================= Tool Message =================================
Name: send_email

Email sent to design-team@example.com - Subject: Reminder: Please Review the New Mockups
================================== Ai Message ==================================

I have sent a reminder to the design team asking them to review the new mockups and provide their feedback. If you need any follow-up or wish to add more details, please l

In [18]:
@tool
def schedule_event(request: str) -> str:
    """Schedule calendar events using natural language.

    Use this when the user wants to create, modify, or check calendar appointments.
    Handles date/time parsing, availability checking, and event creation.

    Input: Natural language scheduling request (e.g., 'meeting with design team
    next Tuesday at 2pm')
    """
    result = calendar_agent.invoke({
        "messages": [{"role": "user", "content": request}]
    })
    return result["messages"][-1].text


@tool
def manage_email(request: str) -> str:
    """Send emails using natural language.

    Use this when the user wants to send notifications, reminders, or any email
    communication. Handles recipient extraction, subject generation, and email
    composition.

    Input: Natural language email request (e.g., 'send them a reminder about
    the meeting')
    """
    result = email_agent.invoke({
        "messages": [{"role": "user", "content": request}]
    })
    return result["messages"][-1].text

In [19]:
SUPERVISOR_PROMPT = (
    "You are a helpful personal assistant. "
    "You can schedule calendar events and send emails. "
    "Break down user requests into appropriate tool calls and coordinate the results. "
    "When a request involves multiple actions, use multiple tools in sequence."
)

supervisor_agent = create_agent(
    model,
    tools=[schedule_event, manage_email],
    system_prompt=SUPERVISOR_PROMPT,
)

In [20]:
query = "Schedule a team standup for tomorrow at 9am"

for step in supervisor_agent.stream(
    {"messages": [{"role": "user", "content": query}]}
):
    for update in step.values():
        for message in update.get("messages", []):
            message.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  schedule_event (call_JcIXynarI1yNxUMYZpP1M0Iz)
 Call ID: call_JcIXynarI1yNxUMYZpP1M0Iz
  Args:
    request: Schedule a team standup for tomorrow at 9am
================================= Tool Message =================================
Name: schedule_event

The team standup has been scheduled for tomorrow at 9:00 AM. If you would like to add specific attendees or a location, please let me know!
================================== Ai Message ==================================

The team standup has been scheduled for tomorrow at 9:00 AM. If you'd like to add specific attendees or set a location, just let me know!


In [21]:
query = (
    "Schedule a meeting with the design team next Tuesday at 2pm for 1 hour, "
    "and send them an email reminder about reviewing the new mockups."
)

for step in supervisor_agent.stream(
    {"messages": [{"role": "user", "content": query}]}
):
    for update in step.values():
        for message in update.get("messages", []):
            message.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  schedule_event (call_DvzjVEmRfZ5uuI92QJrFvS6J)
 Call ID: call_DvzjVEmRfZ5uuI92QJrFvS6J
  Args:
    request: Schedule a meeting with the design team next Tuesday at 2pm for 1 hour.
  manage_email (call_clKP2LtFgHvSEsw6t6I8MIUW)
 Call ID: call_clKP2LtFgHvSEsw6t6I8MIUW
  Args:
    request: Send the design team an email reminder about reviewing the new mockups.
================================= Tool Message =================================
Name: manage_email

I have sent an email to the design team reminding them to review the new mockups. The subject line was "Reminder: Review of New Mockups Needed." Let me know if you need anything else!
================================= Tool Message =================================
Name: schedule_event

The meeting with the design team is scheduled for next Tuesday, June 18th, from 2:00pm to 3:00pm.
================================== Ai Message =============

In [ ]:
from langchain.agents import create_agent
from langchain.agents.middleware import HumanInTheLoopMiddleware 
from langgraph.checkpoint.memory import InMemorySaver 


calendar_agent = create_agent(
    model,
    tools=[create_calendar_event, get_available_time_slots],
    system_prompt=CALENDAR_AGENT_PROMPT,
    middleware=[ 
        HumanInTheLoopMiddleware( 
            interrupt_on={"create_calendar_event": True}, 
            description_prefix="Calendar event pending approval", 
        ), 
    ], 
)

email_agent = create_agent(
    model,
    tools=[send_email],
    system_prompt=EMAIL_AGENT_PROMPT,
    middleware=[ 
        HumanInTheLoopMiddleware( 
            interrupt_on={"send_email": True}, 
            description_prefix="Outbound email pending approval", 
        ), 
    ], 
)

supervisor_agent = create_agent(
    model,
    tools=[schedule_event, manage_email],
    system_prompt=SUPERVISOR_PROMPT,
    checkpointer=InMemorySaver(), 
)

SyntaxError: invalid syntax (2191257957.py, line 35)

In [24]:
query = (
    "Schedule a meeting with the design team next Tuesday at 2pm for 1 hour, "
    "and send them an email reminder about reviewing the new mockups."
)

config = {"configurable": {"thread_id": "6"}}

interrupts = []
for step in supervisor_agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    config,
):
    for update in step.values():
        if isinstance(update, dict):
            for message in update.get("messages", []):
                message.pretty_print()
        else:
            interrupt_ = update[0]
            interrupts.append(interrupt_)
            print(f"\nINTERRUPTED: {interrupt_.id}")

================================== Ai Message ==================================
Tool Calls:
  schedule_event (call_xnwIPo8wzbgM9Tvby6ulNzoo)
 Call ID: call_xnwIPo8wzbgM9Tvby6ulNzoo
  Args:
    request: Schedule a meeting with the design team next Tuesday at 2pm for 1 hour.
  manage_email (call_Ek81KUKrbK5w94DHkDQBpDMY)
 Call ID: call_Ek81KUKrbK5w94DHkDQBpDMY
  Args:
    request: Send an email to the design team reminding them to review the new mockups before our meeting next Tuesday at 2pm.

INTERRUPTED: 441024118f896e1d3c62ce7ffc20f440

INTERRUPTED: ecf1242558958ca630ece114eb86fd78


In [25]:
for interrupt_ in interrupts:
    for request in interrupt_.value["action_requests"]:
        print(f"INTERRUPTED: {interrupt_.id}")
        print(f"{request['description']}\n")

INTERRUPTED: 441024118f896e1d3c62ce7ffc20f440
Outbound email pending approval

Tool: send_email
Args: {'to': ['design.team@example.com'], 'subject': 'Reminder: Please Review New Mockups Before Tuesday’s 2pm Meeting', 'body': 'Hi Team,\n\nThis is a reminder to please review the new mockups prior to our meeting next Tuesday at 2pm. Your feedback will be valuable for our discussion.\n\nThank you!\n\nBest regards,\n[Your Name]'}

INTERRUPTED: ecf1242558958ca630ece114eb86fd78
Calendar event pending approval

Tool: create_calendar_event
Args: {'title': 'Meeting with Design Team', 'start_time': '2024-06-18T14:00:00', 'end_time': '2024-06-18T15:00:00', 'attendees': ['design team']}



In [26]:
from langgraph.types import Command 

resume = {}
for interrupt_ in interrupts:
    if interrupt_.id == "2b56f299be313ad8bc689eff02973f16":
        # Edit email
        edited_action = interrupt_.value["action_requests"][0].copy()
        edited_action["arguments"]["subject"] = "Mockups reminder"
        resume[interrupt_.id] = {
            "decisions": [{"type": "edit", "edited_action": edited_action}]
        }
    else:
        resume[interrupt_.id] = {"decisions": [{"type": "approve"}]}

interrupts = []
for step in supervisor_agent.stream(
    Command(resume=resume), 
    config,
):
    for update in step.values():
        if isinstance(update, dict):
            for message in update.get("messages", []):
                message.pretty_print()
        else:
            interrupt_ = update[0]
            interrupts.append(interrupt_)
            print(f"\nINTERRUPTED: {interrupt_.id}")

================================= Tool Message =================================
Name: manage_email

I've sent an email to the design team reminding them to review the new mockups before the meeting next Tuesday at 2pm. Let me know if you need any further communication!
================================= Tool Message =================================
Name: schedule_event

Your meeting with the design team is scheduled for next Tuesday, June 18th, from 2:00 PM to 3:00 PM. Let me know if you need any further details or changes!
================================== Ai Message ==================================

Your meeting with the design team is scheduled for next Tuesday, June 18th, from 2:00 PM to 3:00 PM.

An email reminder has been sent to the design team, asking them to review the new mockups before the meeting.

Let me know if you need any further details or help!
